In [3]:
# **공고제목 - title**
# **회사이름 - company_name**
# **디테일 페이지로 가는 주소 - detail_url **
# **마감일 - end_date**
# ** 참고한 플랫폼 이름 - platform_name**
# ** 카테고리 - category_name**
# ** 기술 스택 - stack **
# ** 지역  - region **
# ** 신입/경력 - career **

from selenium import webdriver
from selenium.webdriver import ActionChains
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
from bs4 import BeautifulSoup



##### 마우스 이벤트 시작 #####

# 웹 드라이버 설정
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
driver.get("https://www.jobplanet.co.kr/welcome/index/")
driver.implicitly_wait(5)

# 채용 버튼 클릭
button = WebDriverWait(driver, 5).until(
    EC.element_to_be_clickable((By.CLASS_NAME, "job_posting"))
)
ActionChains(driver).click(button).perform()

# 직종 버튼 클릭
job_button = WebDriverWait(driver, 5).until(
    EC.element_to_be_clickable((By.XPATH, "//a[contains(text(), '직종')]"))
)
ActionChains(driver).click(job_button).perform()

# 데이터 버튼 클릭
data_button = WebDriverWait(driver, 5).until(
    EC.element_to_be_clickable((By.XPATH, "//button[text()='데이터']"))
)
ActionChains(driver).click(data_button).perform()

# 데이터 전체 버튼 클릭
engineer_button = WebDriverWait(driver, 5).until(
    EC.element_to_be_clickable((By.XPATH, "//span[text()='데이터 전체']"))
)
ActionChains(driver).click(engineer_button).perform()

# 적용 버튼 클릭
apply_button = WebDriverWait(driver, 5).until(
    EC.element_to_be_clickable((By.XPATH, "//button[text()='적용']"))
)
ActionChains(driver).click(apply_button).perform()

##### 마우스 이벤트 종료 #####

##### 스크랩 시작 #####

# 페이지가 완전히 로드될 때까지 잠시 대기
time.sleep(10)

# 스크롤을 일정 횟수만큼 수행 (예: 5번)
scroll_pause_time = 1
scroll_limit = 5

for _ in range(scroll_limit):
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(scroll_pause_time)

# BeautifulSoup으로 페이지 파싱
html = driver.page_source
soup = BeautifulSoup(html, "html.parser")

# 1. a 태그에서 href 속성을 추출
links = soup.find_all("a", "group z-0 block medium", title="페이지 이동")

# 2. 공고 정보 추출
job_elements = soup.find_all("div", "group mt-[16px] group-[.small]:mt-[14px] medium")

# 3. 결과를 저장할 리스트
jobs_data = []

# 4. 링크와 공고 정보를 함께 출력
for job, link in zip(job_elements, links):
    # 각 공고 정보를 저장할 딕셔너리
    job_data = {}
    
    # 제목 (h2 태그 사용)
    title = job.find("h2", "line-clamp-2 break-all text-h7 text-gray-800 group-[.small]:text-h8")
    if title:
        job_data["title"] = title.get_text(strip=True)

    # 회사명 (em 태그 사용)
    company_name = job.find("em", "inline-block w-full truncate text-body2 font-medium text-gray-800")
    if company_name:
        job_data["company_name"] = company_name.get_text(strip=True)

    # 기술 스택 (span 태그 사용)
    skills = job.find("span", "mt-[6px] inline-block w-full truncate text-small1 text-gray-500")
    if skills:
        job_data["skills"] = skills.get_text(strip=True)

    # 링크 (a 태그에서 href 속성 추출)
    job_data["detail_url"] = link.get("href")

    # 마감일 (임의로 상시 채용으로 설정)
    job_data["end_date"] = "상시 채용"

    # 플랫폼 이름
    job_data["platform_name"] = "Jobplanet"

    # 저장된 공고 데이터를 리스트에 추가
    jobs_data.append(job_data)

# 결과 출력 (예시)
for job in jobs_data:
    print(job)

# 5. 브라우저 종료
driver.quit()

{'title': 'Data Engineer', 'company_name': '(주)자비스앤빌런즈', 'skills': '4년 이상, Spark', 'detail_url': 'https://www.jobplanet.co.kr/job/search?posting_ids%5B%5D=1284279', 'end_date': '상시 채용', 'platform_name': 'Jobplanet'}
{'title': 'Sr. Data Engineer', 'company_name': '잡플래닛', 'skills': '5년 이상, 데이터분석, python, aws, SQL, azure, 머신러닝, nosql, Spark, gcp, Tensorflow', 'detail_url': 'https://www.jobplanet.co.kr/job/search?posting_ids%5B%5D=1269623', 'end_date': '상시 채용', 'platform_name': 'Jobplanet'}
{'title': '특허 빅데이터분석가 (IP빅데이터분석팀)', 'company_name': '(주)워트인텔리전스', 'skills': '3년 이상, 데이터분석, 빅데이터 분석', 'detail_url': 'https://www.jobplanet.co.kr/job/search?posting_ids%5B%5D=1270644', 'end_date': '상시 채용', 'platform_name': 'Jobplanet'}
{'title': 'Machine Learning Engineer, R&D', 'company_name': '(주)클로버추얼패션', 'skills': '2년 이상, 3d, 머신러닝, Docker, 딥러닝', 'detail_url': 'https://www.jobplanet.co.kr/job/search?posting_ids%5B%5D=1276446', 'end_date': '상시 채용', 'platform_name': 'Jobplanet'}
{'title': 'Senior Machine

{
'title': '특허 빅데이터분석가 (IP빅데이터분석팀)',
'company_name': '(주)워트인텔리전스',
'skills': '3년 이상, 데이터분석,빅데이터 분석',
'detail_url': 'https://www.jobplanet.co.kr/job/search?posting_ids%5B%5D=1270644', 
'end_date': '상시 채용', 
'platform_name': 'Jobplanet', 
'stack': ['2024.10.26상시 채용', '빅데이터 엔지니어', '3년 이상', '정규직', '서울', '데이터분석, 빅데이터 분석'], 
'career': '2024.10.26상시 채용', 
'region': '서울'
}

{'title': '특허 빅데이터분석가 (IP빅데이터분석팀)', 'company_name': '(주)워트인텔리전스', 'skills': '3년 이상, 데이터분석, 빅데이터 분석', 'detail_url': 'https://www.jobplanet.co.kr/job/search?posting_ids%5B%5D=1270644', 'end_date': '상시 채용', 'platform_name': 'Jobplanet', 'stack': [], 'career': None, 'region': '서울'}
{'title': '[Riiid] Senior ML Engineer(5년 이상)', 'company_name': '(주)뤼이드', 'skills': '5년 이상,AI, 머신러닝', 'detail_url': 'https://www.jobplanet.co.kr/job/search?posting_ids%5B%5D=1284356', 'end_date': '상시 채용', 'platform_name': 'Jobplanet', 'stack': [], 'career': None, 'region': ' 서울'}
{'title': 'Machine Learning Engineer, R&D', 'company_name': '(주)클로버추얼패션', 'skills': '2년 이상, 3d, 머신러닝, Docker, 딥러닝', 'detail_url': 'https://www.jobplanet.co.kr/job/search?posting_ids%5B%5D=1276446', 'end_date': '상시 채용', 'platform_name': 'Jobplanet', 'stack': [], 'career': None, 'region': '서울'}
{'title': 'Data Engineer', 'company_name': '(주)자비스앤빌런즈', 'skills': '4년 이상, Spark', 'detail_url': 'https://www.jobplanet.co.kr/job/search?posting_ids%5B%5D=1284279', 'end_date': '상시 채용', 'platform_name': 'Jobplanet', 'stack': [], 'career': None, 'region': '서울'}
{'title': 'Sr. Data Engineer', 'company_name': '잡플래닛', 'skills': '5년 이상, 데이터분석, python, aws, SQL, azure, 머신러닝, nosql, Spark, gcp, Tensorflow', 'detail_url': 'https://www.jobplanet.co.kr/job/search?posting_ids%5B%5D=1269623', 'end_date': '상시 채용', 'platform_name': 'Jobplanet', 'stack': ['데이터분석, python, aws, SQL, azure, 머신러닝, nosql, Spark, gcp, Tensorflow'], 'career': None, 'region': '서울'}
{'title': 'Senior Machine Learning Engineer', 'company_name': '(주)몰로코', 'skills': '4년 이상, java, python, machine learning, Tensorflow, Golang', 'detail_url': 'https://www.jobplanet.co.kr/job/search?posting_ids%5B%5D=1278093', 'end_date': '상시 채용', 'platform_name': 'Jobplanet', 'stack': ['java, python, machine learning, Tensorflow, Golang'], 'career': None, 'region': '서울'}
{'title': 'AI 엔지니어 (전문연구요원 신규/편입 가능)', 'company_name': '(주)워트인텔리전스', 'skills': '3년 이상, java, python, 머신러닝, 전문연구요원, 자연어처리, 알고리즘, 성능평가, 기술연구, LLM, MLOPS', 'detail_url': 'https://www.jobplanet.co.kr/job/search?posting_ids%5B%5D=1281619', 'end_date': '상시 채용', 'platform_name': 'Jobplanet', 'stack': ['java, python, 머신러닝, 전문연구요원, 자연어처리, 알고리즘, 성능평가, 기술연구, LLM, MLOPS'], 'career': None, 'region': '서울'}
{'title': 'Data Analytics Engineer(Jr)', 'company_name': '(주)데이터라이즈', 'skills': '1년 이상, python, aws, SQL, 파이썬', 'detail_url': 'https://www.jobplanet.co.kr/job/search?posting_ids%5B%5D=1279583', 'end_date': '상시 채용', 'platform_name': 'Jobplanet', 'stack': ['python, aws, SQL, 파이썬'], 'career': None, 'region': '서울'}
{'title': 'Data Engineer', 'company_name': '(주)데이터라이즈', 'skills': '3년 이상, python, 프로그래밍, SQL, 데이터엔지니어, 데이터수집, 전처리', 'detail_url': 'https://www.jobplanet.co.kr/job/search?posting_ids%5B%5D=1277257', 'end_date': '상시 채용', 'platform_name': 'Jobplanet', 'stack': ['python, 프로그래밍, SQL, 데이터엔지니어, 데이터수집, 전처리'], 'career': None, 'region': '서울'}
{'title': 'Data Engineer(Jr)', 'company_name': '(주)데이터라이즈', 'skills': '1년 이상, python, 프로그래밍, SQL, 데이터엔지니어, 데이터수집, 전처리', 'detail_url': 'https://www.jobplanet.co.kr/job/search?posting_ids%5B%5D=1279584', 'end_date': '상시 채용', 'platform_name': 'Jobplanet', 'stack': ['python, 프로그래밍, SQL, 데이터엔지니어, 데이터수집, 전처리'], 'career': None, 'region': '서울'}
{'title': '데이터사이언티스트 팀장', 'company_name': '(주)해줌', 'skills': '5년 이상, python, 머신러닝, 팀관리, 알고리즘 개발', 'detail_url': 'https://www.jobplanet.co.kr/job/search?posting_ids%5B%5D=1281678', 'end_date': '상시 채용', 'platform_name': 'Jobplanet', 'stack': ['python, 머신러닝, 팀관리, 알고리즘 개발'], 'career': None, 'region': '서울'}
{'title': '[윌로그] 데이터 분석 담당자 리드(10년 이상)', 'company_name': '(주)윌로그', 'skills': '10년 이상, 데이터분석, python, SQL, 데이터모델링, 인사이트 도출', 'detail_url': 'https://www.jobplanet.co.kr/job/search?posting_ids%5B%5D=1278333', 'end_date': '상시 채용', 'platform_name': 'Jobplanet', 'stack': ['데이터분석, python, SQL, 데이터모델링, 인사이트 도출'], 'career': None, 'region': '서울'}
{'title': '데이터 분석가', 'company_name': '(주)브릭', 'skills': '5년 이상, python, R, machine learning, EDA, MLOPS', 'detail_url': 'https://www.jobplanet.co.kr/job/search?posting_ids%5B%5D=1280677', 'end_date': '상시 채용', 'platform_name': 'Jobplanet', 'stack': ['python, R, machine learning, EDA, MLOPS'], 'career': None, 'region': '경기'}
{'title': '[디지털헬스케어] AI 엔지니어 (병역특례 / CTO 레벨 가능)', 'company_name': '(주)엑소시스템즈', 'skills': '경력무관, C++, AI, 데이터분석, python, 딥러닝, 알고리즘 개발', 'detail_url': 'https://www.jobplanet.co.kr/job/search?posting_ids%5B%5D=1280152', 'end_date': '상시 채용', 'platform_name': 'Jobplanet', 'stack': ['C++, AI, 데이터분석, python, 딥러닝, 알고리즘 개발'], 'career': None, 'region': '경기'}
{'title': 'Optimization Research Engineer', 'company_name': '(주)알지에이아이엔씨', 'skills': '경력무관, C++, AI, python, machine learning, Deep Learning, Tensorflow, Pytorch', 'detail_url': 'https://www.jobplanet.co.kr/job/search?posting_ids%5B%5D=1278634', 'end_date': '상시 채용', 'platform_name': 'Jobplanet', 'stack': ['C++, AI, python, machine learning, Deep Learning, Tensorflow, Pytorch'], 'career': None, 'region': '서울'}
{'title': 'AI Language Model Engineer', 'company_name': '(주)알지에이아이엔씨', 'skills': '경력무관, C++, AI, python, machine learning, Deep Learning, Tensorflow, Pytorch', 'detail_url': 'https://www.jobplanet.co.kr/job/search?posting_ids%5B%5D=1278636', 'end_date': '상시 채용', 'platform_name': 'Jobplanet', 'stack': ['C++, AI, python, machine learning, Deep Learning, Tensorflow, Pytorch'], 'career': None, 'region': '서울'}
{'title': 'Research Scientist (Medical AI)', 'company_name': '(주)에이아이트릭스', 'skills': '5년 이상, 머신러닝, machine learning, 딥러닝, Deep Learning', 'detail_url': 'https://www.jobplanet.co.kr/job/search?posting_ids%5B%5D=1278565', 'end_date': '상시 채용', 'platform_name': 'Jobplanet', 'stack': [], 'career': None, 'region': '서울'}
{'title': 'Machine Learning Engineer (전문연구요원), R&D', 'company_name': '(주)클로버추얼패션', 'skills': '2년 이상, 3d, 머신러닝, Docker, 딥러닝', 'detail_url': 'https://www.jobplanet.co.kr/job/search?posting_ids%5B%5D=1276447', 'end_date': '상시 채용', 'platform_name': 'Jobplanet', 'stack': [], 'career': None, 'region': '서울'}
{'title': '데이터 엔지니어', 'company_name': '(주)로민', 'skills': '3년 이상, python, PostgreSQL, 데이터관리, 성과분석, Docker, 시스템구축, airflow, boto3', 'detail_url': 'https://www.jobplanet.co.kr/job/search?posting_ids%5B%5D=1283089', 'end_date': '상시 채용', 'platform_name': 'Jobplanet', 'stack': ['python, PostgreSQL, 데이터관리, 성과분석, Docker, 시스템구축, airflow, boto3'], 'career': None, 'region': '서울'}
{'title': 'Data Scientist', 'company_name': '(주)에이비일팔공', 'skills': '경력무관, python, SQL, R, 머신러닝, machine learning, Tensorflow, Pytorch', 'detail_url': 'https://www.jobplanet.co.kr/job/search?posting_ids%5B%5D=1282238', 'end_date': '상시 채용', 'platform_name': 'Jobplanet', 'stack': ['python, SQL, R, 머신러닝, machine learning, Tensorflow, Pytorch'], 'career': None, 'region': '서울'}
{'title': 'AI 컴파일러 엔지니어 (전문연구요원 가능)', 'company_name': '(주)에너자이', 'skills': '경력무관, c, C++, AI, 알고리즘', 'detail_url': 'https://www.jobplanet.co.kr/job/search?posting_ids%5B%5D=1280545', 'end_date': '상시 채용', 'platform_name': 'Jobplanet', 'stack': [], 'career': None, 'region': '서울'}
{'title': '시니어 데이터 사이언티스트', 'company_name': '(주)아이넥스코퍼레이션', 'skills': '5년 이상, C++, python, Linux, 딥러닝, opencv', 'detail_url': 'https://www.jobplanet.co.kr/job/search?posting_ids%5B%5D=1280951', 'end_date': '상시 채용', 'platform_name': 'Jobplanet', 'stack': ['C++, python, Linux, 딥러닝, opencv'], 'career': None, 'region': '서울'}
{'title': 'Robot Perception Engineer', 'company_name': '(주)알지에이아이엔씨', 'skills': '경력무관, C++, AI, python, machine learning, Deep Learning, Tensorflow, Pytorch', 'detail_url': 'https://www.jobplanet.co.kr/job/search?posting_ids%5B%5D=1278632', 'end_date': '상시 채용', 'platform_name': 'Jobplanet', 'stack': ['C++, AI, python, machine learning, Deep Learning, Tensorflow, Pytorch'], 'career': None, 'region': '서울'}
{'title': 'Staff Machine Learning Engineer', 'company_name': '(주)몰로코', 'skills': '7년 이상, java, python, machine learning, Tensorflow, Golang', 'detail_url': 'https://www.jobplanet.co.kr/job/search?posting_ids%5B%5D=1278094', 'end_date': '상시 채용', 'platform_name': 'Jobplanet', 'stack': ['java, python, machine learning, Tensorflow, Golang'], 'career': None, 'region': '서울'}

In [4]:
from selenium import webdriver
from selenium.webdriver import ActionChains
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
from bs4 import BeautifulSoup
from concurrent.futures import ThreadPoolExecutor, as_completed

## 스킬태그
# <span class="Typography_Typography__root__RdAI1 Typography_Typography__label2__svmAA Typography_Typography__weightMedium__GXnOM">Python</span> 
# <li class="SkillTagItem_SkillTagItem__K3B3t"><span class="Typography_Typography__root__RdAI1 Typography_Typography__label2__svmAA Typography_Typography__weightMedium__GXnOM">Tableau</span></li> 
# <strong class="JobCard_title__HBpZf">[Data Exchange]Project Manager</strong> # title 태그
# <span class="JobCard_companyName__N1YrF">앤서스랩코리아</span> # company_name 태그


import requests
from bs4 import BeautifulSoup

# 웹 페이지 요청
url = "https://www.wanted.co.kr/search?query=%EB%8D%B0%EC%9D%B4%ED%84%B0&tab=position"  # 실제 URL로 변경해야 합니다.
response = requests.get(url)
print(response.text.pr)

<!DOCTYPE html><html lang="ko-KR" class="ko kr"><head><meta charSet="utf-8"/><meta name="viewport" content="width=device-width, initial-scale=1.0"/><meta http-equiv="X-UA-Compatible" content="IE=edge"/><meta http-equiv="cache-control" content="no-store"/><meta property="og:type" content="website"/><meta property="og:locale" content="ko_KR"/><meta property="fb:app_id" content="316787678519888"/><meta name="theme-color" content="#fffffe"/><meta name="msapplication-TileColor" content="#ffffff"/><meta name="msapplication-TileImage" content="//static.wanted.co.kr/favicon/new/144x144.png"/><meta name="naver-site-verification" content="ef4f7f8504106401fe915b922514a5b352f59ca2"/><meta name="google-site-verification" content="KEXLzgQBOVNqw8uNBWOXYOz2L4vUApbKxqxV6twdkxQ"/><title>데이터 | 원티드 통합검색</title><meta property="og:title" content="데이터 | 원티드 통합검색"/><meta name="description" content="데이터 검색 결과를 지금 원티드에서 확인하세요."/><meta property="og:description" content="데이터 검색 결과를 지금 원티드에서 확인하세요."/><meta propert